# Test search utils


In [2]:
import yaml
import inspect
import importlib
import os,sys
import importlib

sys.path.append('..')

from model_discovery.evolution import BuildEvolution
import model_discovery.utils as U




strparams=[
    f"evoname=test_evo_000",
    "scales=14M,31M,70M",
    "selection_ratio=0.25",
    "select_method=random",
    "design_budget=0",
]
evo_system = BuildEvolution(
    strparams=';'.join(strparams),
    do_cache=False,
    # cache_type='diskcache',
)

ptree = evo_system.ptree


c:\Users\ChengJunyan1\anaconda3\envs\modis\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to C:\Users\ChengJunyan1\.cache\huggingface\token
Login successful


2024-09-11:16:55:25,786 INFO     [model_agent.py:184] Agent name=`designer`, model_details={
    "model_name": "gpt-4o-2024-08-06",
    "max_output_tokens": 8192,
    "temperature": 0.3
}


Loaded tokenized dataset wikitext-2 from C:\ChengJunyan1\Research\model_discovery\data\wikitext-2\tokenized\meta-llama/Llama-2-7b-hf\2048


2024-09-11:16:55:26,848 INFO     [model_agent.py:184] Agent name=`debugger`, model_details={
    "model_name": "gpt-4o-mini",
    "max_output_tokens": 8192
}
2024-09-11:16:55:27,153 INFO     [model_agent.py:184] Agent name=`claude`, model_details={
    "model_name": "gpt-4o",
    "max_output_tokens": 8192
}


{'evoname': 'test_evo_000', 'scales': '14M,31M,70M', 'selection_ratio': '0.25', 'select_method': 'random', 'design_budget': '0'}
Evolution system initialized with scales: ['14M', '31M', '70M']
Current scale: 0
Budgets remaining: {'70M': 1, '31M': 4, '14M': 16}
Checkpoint directory: C:\ChengJunyan1\Research\model_discovery\ckpt\test_evo_000
gpt2 tree loaded


2024-09-11:16:55:32,841 INFO     [model_agent.py:184] Agent name=`designer`, model_details={
    "model_name": "gpt-4o-2024-08-06",
    "max_output_tokens": 8192,
    "temperature": 0.3
}


Loaded tokenized dataset wikitext-2 from C:\ChengJunyan1\Research\model_discovery\data\wikitext-2\tokenized\meta-llama/Llama-2-7b-hf\2048


2024-09-11:16:55:33,557 INFO     [model_agent.py:184] Agent name=`debugger`, model_details={
    "model_name": "gpt-4o-mini",
    "max_output_tokens": 8192
}
2024-09-11:16:55:33,834 INFO     [model_agent.py:184] Agent name=`claude`, model_details={
    "model_name": "gpt-4o",
    "max_output_tokens": 8192
}


In [3]:

refs_s2ids={}
refs_s2ids_1hop={}
REF_TYPES=['Reference','ReferenceCore','ReferenceWithCode','ReferenceCoreWithTree']
REF_TYPES_1HOP=['Reference1hop']    

for i in ptree.G.nodes:
    node=ptree.G.nodes[i]['data']
    if node.type in REF_TYPES:
        s2id=node.s2id
        if s2id is not None:
            refs_s2ids[i]=s2id
        else:
            print(i)
    elif node.type in REF_TYPES_1HOP:
        s2id=node.s2id
        if s2id is not None:
            refs_s2ids_1hop[i]=s2id
        else:
            print(i)

refs=ptree.filter_by_type(REF_TYPES)
refs_1hop=ptree.filter_by_type(REF_TYPES_1HOP)

feedbackmem
kangpt
ntk
s4pp
srt


In [4]:
import requests
import time
from tqdm import tqdm

headers={"X-API-KEY": os.environ['S2_API_KEY']}
paper_detail='https://api.semanticscholar.org/graph/v1/paper/{paper_id}'
params={
    "fields": "openAccessPdf,externalIds",
}

# # pdf_urls={}
# external_ids={}
# for i in tqdm(refs_s2ids):
#     s2id=refs_s2ids[i]
#     rsp=requests.get(
#         paper_detail.format(paper_id=s2id),
#         headers=headers,
#         params=params
#     )
#     results = rsp.json()
#     if 'openAccessPdf' in results and results['openAccessPdf'] is not None:
#         pdf_urls[i]=results['openAccessPdf']
#     if 'externalIds' in results and results['externalIds'] is not None:
#         external_ids[i]=results['externalIds']
#     time.sleep(0.1)


# arxiv_ids={}
# for i,j in external_ids.items():
#     if 'ArXiv' in j:
#         arxiv_ids[i]=j['ArXiv']


# pdf_urls_1hop={}
# external_ids_1hop={}
# for i in tqdm(refs_s2ids_1hop):
#     s2id=refs_s2ids_1hop[i]
#     rsp=requests.get(
#         paper_detail.format(paper_id=s2id),
#         headers=headers,
#         params=params
#     )
#     results = rsp.json()
#     if 'openAccessPdf' in results and results['openAccessPdf'] is not None:
#         pdf_urls_1hop[i]=results['openAccessPdf']
#     if 'externalIds' in results and results['externalIds'] is not None:
#         external_ids_1hop[i]=results['externalIds']
#     time.sleep(0.1)


# arxiv_ids_1hop={}
# for i,j in external_ids_1hop.items():
#     if 'ArXiv' in j:
#         arxiv_ids_1hop[i]=j['ArXiv']




In [7]:
def get_node(id):
    return ptree.G.nodes[id]['data']


def get_references(id):
    params={
        "fields": "references",
    }
    s2id=refs_s2ids[id]
    rsp=requests.get(
        paper_detail.format(paper_id=s2id),
        headers=headers,
        params=params
    )
    results = rsp.json()
    return results['references']


all_refs={}
for i in tqdm(refs_s2ids):
    all_refs[i]=get_references(i)

refs_set={}
for i in all_refs:
    for ref in all_refs[i]:
        refs_set[ref['paperId']]=ref['title']


100%|██████████| 292/292 [00:33<00:00,  8.79it/s]


In [8]:
def get_paper_detail(s2id):
    params={
        "fields": "title,abstract,venue,year,referenceCount,citationCount,influentialCitationCount,authors,tldr,embedding,tldr,openAccessPdf,externalIds,references",
    }
    rsp=requests.get(
        paper_detail.format(paper_id=s2id),
        headers=headers,
        params=params
    )
    results = rsp.json()
    return results

refs_detail={}
save_dir=U.pjoin(ptree.lib_dir,'..','tree_ext','refs2')
for i in tqdm(refs_set):
    save_path=U.pjoin(save_dir,f'{i}.json')
    if U.pexists(save_path):
        refs_detail[i]=U.load_json(save_path)
        continue
    ret=get_paper_detail(i)
    if 'error' not in ret:
        U.save_json(ret,save_path)
    refs_detail[i]=ret
    time.sleep(0.1)


100%|██████████| 4775/4775 [00:21<00:00, 218.05it/s] 


In [9]:
len(refs_detail)

ref2_arxiv={}
ref2_pdfs={}
for i in refs_detail:
    if 'externalIds' in refs_detail[i]:
        if 'ArXiv' in refs_detail[i]['externalIds']:
            ref2_arxiv[i]=refs_detail[i]['externalIds']['ArXiv']
            continue
    if 'openAccessPdf' in refs_detail[i]:
        url=refs_detail[i]['openAccessPdf']
        if url:
            ref2_pdfs[i]=url
        continue
    print(i)
    

None


In [13]:
ref2_arxiv[i]

'2402.17463'

In [14]:
import urllib, urllib.request

save_dir=U.pjoin(ptree.lib_dir,'..','pdfs2')
arxiv_pdf='https://arxiv.org/pdf/{arxiv_id}'
U.mkdir(save_dir)
bar=tqdm(ref2_arxiv)
for i in bar:
    bar.set_description(f'Downloading {i}')
    arxiv_id=ref2_arxiv[i]
    url=arxiv_pdf.format(arxiv_id=arxiv_id)
    save_path=U.pjoin(save_dir,f'{i}.pdf')
    if not os.path.exists(save_path):
        pdf=urllib.request.urlretrieve(url, save_path)


# failed_pdfs={}
# for i in tqdm(ref2_pdfs):
#     url=ref2_pdfs[i]['url']
#     save_path=U.pjoin(save_dir,f'{i}.pdf')
#     if not os.path.exists(save_path):
#         try:
#             pdf=urllib.request.urlretrieve(url, save_path)
#         except:
#             failed_pdfs[i]=url
#             print(f'Failed to download {i}')


 12%|█▏        | 390/3172 [00:09<01:08, 40.84it/s]


KeyboardInterrupt: 

In [16]:
pdf[1]

In [82]:
for i in refs:
    if i not in arxiv_ids:
        print(i)


clusterformer
etc
feedbackmem
fire
gpt2
hedgehog
infiniteformer
kangpt
mogrifier
ntk
remixer
s4pp
settransformer
sparsetokentrans
srt
synthesizer


In [78]:
save_dir

'c:\\ChengJunyan1\\Research\\model_discovery\\notebooks\\..\\model_discovery\\model\\library\\tree\\pdfs'

In [57]:
import urllib, urllib.request
url = 'http://export.arxiv.org/api/query?search_query="{query}"&start=0&max_results=1'
title = ptree.G.nodes[refs[0]]['data'].title
encoded_title = urllib.parse.quote(title)
url=url.format(query=encoded_title)
data = urllib.request.urlopen(url)
print(data.read().decode('utf-8'))


<?xml version="1.0" encoding="UTF-8"?>
<feed xmlns="http://www.w3.org/2005/Atom">
  <link href="http://arxiv.org/api/query?search_query%3D%223D-RPE%3A%20Enhancing%20Long-Context%20Modeling%20Through%203D%20Rotary%20Position%20Encoding%22%26id_list%3D%26start%3D0%26max_results%3D1" rel="self" type="application/atom+xml"/>
  <title type="html">ArXiv Query: search_query="3D-RPE: Enhancing Long-Context Modeling Through 3D Rotary Position Encoding"&amp;id_list=&amp;start=0&amp;max_results=1</title>
  <id>http://arxiv.org/api/WZ9KoGw6d+aii+09GWS4ZE43lX0</id>
  <updated>2024-09-11T00:00:00-04:00</updated>
  <opensearch:totalResults xmlns:opensearch="http://a9.com/-/spec/opensearch/1.1/">0</opensearch:totalResults>
  <opensearch:startIndex xmlns:opensearch="http://a9.com/-/spec/opensearch/1.1/">0</opensearch:startIndex>
  <opensearch:itemsPerPage xmlns:opensearch="http://a9.com/-/spec/opensearch/1.1/">1</opensearch:itemsPerPage>
</feed>



In [46]:
len(pdf_urls)

103

In [ ]:
import agents.search_utils 
importlib.reload(agents.search_utils)

from agents.search_utils import SuperScholarSearcher



